## Boundary Layer Equation Problem

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
# parameters and initialization
M = 100    # number of steps on x axis which is delta x
N = 50     # number of steps on y axis
x = 0.1      # length of x
y = 1        # length of y
dx = x / M   # step size of x axis, we only consider one step 
dy = y / N   # step size of y axis, where y is from 0 to 1

# pressures of the first three
p_0 = 0  #update p_0
p_1 = 0.01
p_2 = 0.02

In [11]:
# Thomas algorithm function
def TDMA(a,b,c,d):
    n = len(d)
    w= np.zeros(n-1,float)
    g= np.zeros(n, float)
    p = np.zeros(n,float)

    w[0] = c[0]/b[0]
    g[0] = d[0]/b[0]

    for i in range(1,n-1):
        w[i] = c[i]/(b[i] - a[i-1]*w[i-1])
    for i in range(1,n):
        g[i] = (d[i] - a[i-1]*g[i-1])/(b[i] - a[i-1]*w[i-1])
    p[n-1] = g[n-1]
    for i in range(n-1,0,-1):
        p[i-1] = g[i-1] - w[i-1]*p[i]
    return p

In [12]:
# function to delete first and last row
def row_delete(A, N):
    A = np.delete(A, 0, axis = 0)
    A = np.delete(A, N, axis = 0)
    return A

In [13]:
# function about tridiagional matrix
def tridiagional_matrix(u, v, p_guess,p_correct, N):
    # main diagional
    main_old = np.zeros(N + 1)
    for i in range(1, N):
        main_old[i] = u[i] / dx + 2 / (dy ** 2)
    main = row_delete(main_old, N - 1)

    # upper diagional
    upper_old = np.zeros(N)
    for i in range(1, N):
        upper_old[i] = (v[i] / (2 * dy) - 1 / (dy ** 2))
    upper = row_delete(upper_old, N - 2)
    
    # lower diagional
    lower_old = np.zeros(N)
    for i in range(1, N):
        lower_old[i] = (- v[i] / (2 * dy) - 1 / (dy ** 2))
    lower = row_delete(lower_old, N - 2)   
    
    # RHS
    RHS_old = np.zeros(N + 1)
    for i in range(1, N - 1):
        RHS_old[i] = u[i] ** 2 / dx - (p_guess - p_correct) / dx
    RHS = row_delete(RHS_old, N - 1)
    return lower, main, upper, RHS

In [14]:
# function to update the velocity of u
def u_next_step(lower, main, upper, RHS):
    sol = np.zeros(N - 1)  # 49
    sol = TDMA(lower,main,upper,RHS)
    u = np.zeros(N + 1)    # 51
    for i in range(1,N):   # 49
        u[i] = sol[i - 1]
    return u

In [15]:
# function to update the velocity of v
def v_next_step(v, u_update, u):
    for i in range(1, N):
        v[i] = v[i - 1] - 0.5 * dy * ((u_update[i] - u[i]) / dx + (u_update[i - 1] - u[i - 1]) / dx)
    return v

In [16]:
# function to more steps 
def iteration(M, N):
    u_current = np.zeros(N + 1)    
    for i in range(1, N):
         u_current[i] = math.sin(math.pi * i * 0.02)
    v_current = np.zeros(N + 1)
    
    u_current_2 = np.zeros(N + 1)    
    for i in range(1, N):
        u_current_2[i] = math.sin(math.pi * i * 0.02)
    v_current_2 = np.zeros(N + 1)
    
    p_correct = 0
    
    for j in range(0, M):
        # get updated u and v with p_guess = 0.01
        coef = tridiagional_matrix(u_current, v_current, 0.01, p_correct, N)
        lower = coef[0]
        main = coef[1]
        upper = coef[2]
        RHS = coef[3]
        u_update = u_next_step(lower, main, upper, RHS)
        v_update = v_next_step(v_current, u_update, u_current)
        u_current = u_update
        v_current = v_update 
        
        # get updated u and v with p_guess = 0.02
        coef_2 = tridiagional_matrix(u_current_2, v_current_2, 0.02, p_correct, N)
        lower_2 = coef_2[0]
        main_2 = coef_2[1]
        upper_2 = coef_2[2]
        RHS_2 = coef_2[3]
        u_update_2 = u_next_step(lower_2, main_2, upper_2, RHS_2)
        v_update_2 = v_next_step(v_current_2, u_update_2, u_current_2)
        u_current_2 = u_update_2
        v_current_2 = v_update_2
        
        # get p_correct
        denom = v_update_2[N-1] - v_update[N-1]
        numer = 0.01*v_update_2[N-1] - 0.02*v_update[N-1]
        p_correct = numer/denom
    return v_current, v_current_2, p_correct